In [2]:
import cv2
import numpy as np

# Load video
video_path = "FNSEIZURE.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# video writer to save output
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('TESToutput_labeled1.avi', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))


fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=16, detectShadows=True)
min_contour_area = 1000
kernel = np.ones((5, 5), np.uint8)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Motion detection using background subtraction
    fgmask = fgbg.apply(frame)
    fgmask = cv2.threshold(fgmask, 200, 255, cv2.THRESH_BINARY)[1]  # Thresholding
    fgmask = cv2.medianBlur(fgmask, 5)  # Apply median blur to reduce noise
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_CLOSE, kernel)  # Morphological closing
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)  # Morphological opening to remove small objects

    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        if cv2.contourArea(contour) > min_contour_area:
            x, y, w, h = cv2.boundingRect(contour)
            roi = frame[y:y+h, x:x+w]
            # Classification logic based on bounding box size
            if h > 200 and w > 100:  # condition for 'walking'
                label = "Walking"
            elif h > 50 and w < 50:  # condition for 'standing'
                label = "Standing"
            else:
                label = "Seizure"

            # Draw bounding box and label
            color = (0, 255, 0) if label == "Walking" else (255, 0, 0) if label == "Standing" else (0, 0, 255)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    
    out.write(frame)

    
    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()


Number of persons detected: 38
